In [20]:
import json
import os
from omero.rtypes import rstring
from omero.gateway import BlitzGateway
from biohack_utils.config_utils import flatten, unflatten, download, upload
from biohack_utils.ConfigSchema import OMEWrapper
from biohack_utils.ConfigSchema import CollectionNode
from biohack_utils.util import connect_to_omero
import dotenv
import pprint


In [22]:
def connect_to_omero():
    dotenv.load_dotenv()
    USERNAME = os.getenv("OMERO_USERNAME")
    PASSWORD = os.getenv("OMERO_PASSWORD")
    HOST = os.getenv("OMERO_HOST")
    PORT = int(os.getenv("OMERO_PORT", 4064))  # Default OMERO port

    conn = BlitzGateway(USERNAME, PASSWORD, host=HOST, port=PORT)
    conn.connect()

    if conn.isConnected():
        print("Connected to OMERO")
    else:
        print("Failed to connect")
        exit(1)

    return conn


In [14]:
dotenv.load_dotenv()
    # Read the json file
json_file_path = "example2.json"
    
with open(json_file_path) as f:
    d = json.load(f)

In [5]:
wrapper = OMEWrapper.model_validate(d)

In [6]:
data_dict = wrapper.model_dump()

# 3. Export with Indentation 💾
file_path = "output_from_wrapper.json"

with open(file_path, 'w', encoding='utf-8') as f:
    # Key Step: Use json.dump() and set indent=4
    # The 'indent=4' parameter tells Python to add line breaks and 4 spaces
    # of indentation for every nested level.
    json.dump(data_dict, f, indent=2)

In [7]:
flat = flatten(wrapper)
pprint.pprint(flat)

[{'category': 'intensities',
  'description': 'Original nuclei channel',
  'name': 'nuclei_raw',
  'omero:image_id': 51,
  'origin': 'raw',
  'path': 'nuclei_raw'},
 {'category': 'annotations',
  'description': 'Nuclei segmentation masks',
  'name': 'nuclei_segmentation',
  'omero:image_id': 54,
  'origin': 'masks',
  'path': 'nuclei_segmentation',
  'source': 'nuclei_raw'},
 {'category': 'annotations',
  'description': 'Cell segmentation masks',
  'name': 'cell_segmentation',
  'omero:image_id': 55,
  'origin': 'masks',
  'path': 'cell_segmentation',
  'source': 'nuclei_raw'}]


In [8]:
# Unflatten example
unflattened = unflatten(flat, "cell_analysis").model_dump()
# 3. Export with Indentation 💾
file_path = "output_from_after_unwrapping.json"

with open(file_path, 'w', encoding='utf-8') as f:
    # Key Step: Use json.dump() and set indent=4
    # The 'indent=4' parameter tells Python to add line breaks and 4 spaces
    # of indentation for every nested level.
    json.dump(unflattened, f, indent=2)

In [18]:
conn = connect_to_omero()

Connected to OMERO


In [19]:
coll_id = upload(conn,wrapper)

In [17]:
schema = download(conn, 75)


Processing image 51
  Found 1 node annotations via query
  ✓ Match found!

Processing image 54
  Found 1 node annotations via query
  ✓ Match found!

Processing image 55
  Found 1 node annotations via query
  ✓ Match found!

Total flat records collected: 3


ValidationError: 1 validation error for NodeAttributes
  Value error, Source required for origin 'masks' [type=value_error, input_value={'omero:image_id': 54, 'c...lei segmentation masks'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
# 3. Export with Indentation
file_path = "output_from_omero.json"

with open(file_path, 'w', encoding='utf-8') as f:

    json.dump(data_dict, f, indent=4)

# To clean up

In [35]:
# Get the image  
image = conn.getObject("Image", 35485)  
  
# Delete all map annotations (by namespace)  
image.removeAnnotations(ns="ome/collection")  # Specify namespace or None for all
image.removeAnnotations(ns="ome/collection/nodes")  # Specify namespace or None for all

 